**IMPORTANT** <br> <ul> <li> Do **NOT** replace or remove this notebook (ipynb file)! Each cell has unique nbgrader's metadata and ID which, if changed outside the nbgrader, cannot pass the tests. Do **NOT** change the name of the file!</li> <li> To receive any credit, don't forget to **SUBMIT** your notebook when you are done! You can have multiple submissions before the deadline; only the last one is saved, including its timestamp.</li> <li>Before submitting, **Validate** your notebook to check if your codes pass all visible tests. </li> <li>Make sure you fill in any cell with the comment `# your code here`. Remove or comment the command `fail()` (in R), or `raise NotImplementedError` (in Python) and place your code there </li> </ul>

In [1]:
NAME = "Madison Chester"

---

# Naive Bayes in Python


<br>

Let's start by importing libraries. Run the following cell.


In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import OrdinalEncoder # for encoding categorical features from strings to number arrays
from sklearn.naive_bayes import MultinomialNB, CategoricalNB

from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

## Read in and quick look at the data 

<br>

We use data set `Heart.csv`, from *An Introduction to Statistical Learning with Applications in R* (ISLR): <https://www.statlearning.com>. <br>

Run the following cell to explore the data set.

In [3]:
df = pd.read_csv('Heart.csv', sep=',')

print(df.shape)
df.head()


(303, 15)


,PatientID,Age,Sex,ChestPain,RestBP,Chol,Fbs,RestECG,MaxHR,ExAng,Oldpeak,Slope,Ca,Thal,AHD
0,1,63,1,typical,145,233,1,2,150,0,2.3,3,0.0,fixed,No
1,2,67,1,asymptomatic,160,286,0,2,108,1,1.5,2,3.0,normal,Yes
2,3,67,1,asymptomatic,120,229,0,2,129,1,2.6,2,2.0,reversable,Yes
3,4,37,1,nonanginal,130,250,0,0,187,0,3.5,3,0.0,normal,No
4,5,41,0,nontypical,130,204,0,2,172,0,1.4,1,0.0,normal,No


Each row (i.e. observation) represents a single patient. The last variable, `AHD`, is binary (Yes/No) and indicates presence of heart disease based on the **A**ngiographic **H**eart **D**isease test. This variable will be our output categorical variable. All other variables except the first one (PatientID) are our predictors/features.<br>

<br>

### Goal

The goal is to use Naive Bayes method to classify patients by the `AHD` label. That is, based on the predictor variables (features), we want to predict whether the patient has heart disease (`AHD = "Yes"`) or not (`AHD = "No"`).

<br>

## Data munging and cleaning

<br>

The variable `PatientID` is not a predictor, so we remove it from our data frame `df`.

To remove `PatientID`, use `pandas` method `drop()` to drop

`df = df.drop(columns = ??)`

or `iloc()` to extract needed data:

`df = df.iloc[??, ??]`

By now (i.e. after several DAT classes), we are not beginners anymore and should be comfortable with googling documentation for a desireable method/function, or searching for how to perform certain action (eg: "`how to drop/remove column in pandas data frame in python`"). A good portion of their job, data scientist/programmer spends on googling things like this, since one cannot (**and should not**) memorize all the syntax.

In [4]:
df = df.drop(columns=df.columns[0])

print(df.shape)
df.head()

(303, 14)


,Age,Sex,ChestPain,RestBP,Chol,Fbs,RestECG,MaxHR,ExAng,Oldpeak,Slope,Ca,Thal,AHD
0,63,1,typical,145,233,1,2,150,0,2.3,3,0.0,fixed,No
1,67,1,asymptomatic,160,286,0,2,108,1,1.5,2,3.0,normal,Yes
2,67,1,asymptomatic,120,229,0,2,129,1,2.6,2,2.0,reversable,Yes
3,37,1,nonanginal,130,250,0,0,187,0,3.5,3,0.0,normal,No
4,41,0,nontypical,130,204,0,2,172,0,1.4,1,0.0,normal,No


<br>
Next, run the following line to see whether there are missing data in some columns.
<br>

In [5]:
df.isnull().sum(axis=0)  #sum along first dimension/axis, so, indexed by 0 (axis=0)

Age          0
Sex          0
ChestPain    0
RestBP       0
Chol         0
Fbs          0
RestECG      0
MaxHR        0
ExAng        0
Oldpeak      0
Slope        0
Ca           4
Thal         2
AHD          0
dtype: int64

Run the following cell to remove all observations (rows/patients) who have at least one datum missing. (There should be 6 obs removed - compare the shape of `df` before and after the action).

In [6]:
df = df.dropna()
df.shape

(297, 14)

<br>

Let's see the proportions of labels (Yes/No) in `AHD`.

In [7]:
## what ratio of No and Yes
df['AHD'].value_counts(normalize=True)

No     0.538721
Yes    0.461279
Name: AHD, dtype: float64

## Fit Naive Bayes on train, predict on test data


<br>


As discussed in class, we should test performance on a data set not used for training the model. For that purpose, we sacrifize some data and instead of using them for training, we only use them for testing. We split the original data into training and test data at the ratio, say, 70/30.

So, we are going to perform the following 3 steps:

- split the original data into train and test datasets
- make a model and fit it on the train data
- predict on the test data

In [8]:
# set seed
np.random.seed(1234)

# Randomize the dataset (i.e. shuffle the rows); 
data_randomized = df.sample(frac=1)  #could also set seed by argument random_state=1234

# Calculate index for split (size of train data) -take first 70% of the data for training set
trainsize = round(len(data_randomized) * 0.7)

# Split into training and test sets
training_set = data_randomized[:trainsize].reset_index(drop=True)
test_set = data_randomized[trainsize:].reset_index(drop=True)

print(training_set.shape)
print(test_set.shape)

(208, 14)
(89, 14)


#### Sanity check

<br>

If everything is okay, our `training_set` and `test_set` should look structurally the same as the original data frame. We plot them for so called *sanity check*, i.e. to see whether everything looks okay at least at first sight, i.e. without diving much into details.

In [9]:
print(type(training_set))
training_set.head(6)

<class 'pandas.core.frame.DataFrame'>


,Age,Sex,ChestPain,RestBP,Chol,Fbs,RestECG,MaxHR,ExAng,Oldpeak,Slope,Ca,Thal,AHD
0,65,1,asymptomatic,110,248,0,2,158,0,0.6,1,2.0,fixed,Yes
1,39,1,asymptomatic,118,219,0,0,140,0,1.2,2,0.0,reversable,Yes
2,71,0,asymptomatic,112,149,0,0,125,0,1.6,2,0.0,normal,No
3,43,0,asymptomatic,132,341,1,2,136,1,3.0,2,0.0,reversable,Yes
4,52,1,typical,118,186,0,2,190,0,0.0,2,0.0,fixed,No
5,55,1,asymptomatic,140,217,0,0,111,1,5.6,3,0.0,reversable,Yes


<br>

If everything is okay, we should have similar proportions of the two labels (Yes/No) in the training and test sets.

In [10]:
training_set['AHD'].value_counts(normalize=True)

No     0.533654
Yes    0.466346
Name: AHD, dtype: float64

In [11]:
test_set['AHD'].value_counts(normalize=True)

No     0.550562
Yes    0.449438
Name: AHD, dtype: float64

## Creating the Naive Bayes model

<br>

First, we create pandas data frame `trainX` and  pandas series (vector) `trainy`. The data frame `trainX` should consist of only features/predictors of the training data, not the output `AHD`. On the other hand, `trainy` should be the corresponding labels (i.e. column `AHD`) of the training data.

After creating them, print out first couple of observations for each of the two objects, to check whether everything is okay (sanity check).

In [12]:
trainX = training_set.iloc[:,:-1]
trainy = training_set['AHD']

colnames = trainX.columns

trainX.head()

trainy.head()

0    Yes
1    Yes
2     No
3    Yes
4     No
Name: AHD, dtype: object

In [13]:
test_set.head()

,Age,Sex,ChestPain,RestBP,Chol,Fbs,RestECG,MaxHR,ExAng,Oldpeak,Slope,Ca,Thal,AHD
0,62,0,asymptomatic,160,164,0,2,145,0,6.2,3,3.0,reversable,Yes
1,48,1,asymptomatic,122,222,0,2,186,0,0.0,1,0.0,normal,No
2,51,1,nonanginal,100,222,0,0,143,1,1.2,2,0.0,normal,No
3,63,1,typical,145,233,1,2,150,0,2.3,3,0.0,fixed,No
4,59,1,asymptomatic,140,177,0,0,162,1,0.0,1,1.0,reversable,Yes


### <font style="color: red">Question 1.</font>

<br>

Now, it's your turn. Create pandas data frame `testX` and pandas series (vector) `testy`. The data frame `testX` should consist of all the features/predictors values from the test data and without the output `AHD`. The vector (i.e. pandas series) `testy` should only consist of output values (i.e. labels) from the test data. 

In [14]:
testX = test_set.iloc[:,:-1]
testy = test_set['AHD']

In [15]:
## test whether testX is pandas data frame
assert isinstance(testX, pd.core.frame.DataFrame)

## test whether first 4 values of column Age are correct
assert sum(testX['Age'][:4] != [62, 48, 51, 63]) == 0

## test whether first 4 values of column ChestPain are correct
assert sum(testX['ChestPain'][:4] != ['asymptomatic','asymptomatic','nonanginal','typical']) == 0

## test whether first 4 values of column Chol are correct
assert np.linalg.norm(np.array(testX['Chol'][:4]) - np.array([164, 222, 222, 233]), ord=2) < 1.e-5

In [16]:
## test whether testy is pandas series
assert isinstance(testy, pd.core.series.Series)


## test first 4 and last 4 values of testy are correct
assert sum(testy.iloc[:4] != ['Yes', 'No', 'No', 'No']) == 0
assert sum(testy.iloc[-4:] != ['Yes', 'No', 'Yes','Yes']) == 0

### Encoding and training

To prepare for training, we encode train labels (`trainy`) into 0-1 codes (Bernoulli random variable). The 0-1 coded values are stored in the variable `trainBrnli`.

One way to do this is by typing 

`trainBrnli = np.array([int(trainy[i]=='Yes') for i in range(len(trainy))])`

However, we will use `LabelEncoder()` function from `sklearn.preprocessing` submodule.

In [17]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder() #rename function for simpler writing

#0-1 encoding train labels (think of it as a Bernoulli variable)
trainBrnli = le.fit_transform(trainy)

trainBrnli[:5] #print first 5 of train Bernoulli (check that No=0, Yes=1)

array([1, 1, 0, 1, 0])

<br>

#### Naive Bayes with mixed variables

In our dataset `Heart` we have both discrete and continuous features. Submodule `sklearn.naive_bayes` does not handle this situation automatically. There are several ways to proceed, each simplifying the situation. Probably the simplest one (which we will perform) is binning the numerical features. That is, for each numerical feature, we group its values into bins, and each bin is represented by a single number from 0, 1, ... , n, where n is the number of bins (i.e. categories) for this feature.

To encode the the data from the feature data frame `trainX`, we use `OrdinalEncoder` function from the submodule `sklearn.preprocessing`.

In [18]:
from sklearn.preprocessing import OrdinalEncoder

enc = OrdinalEncoder()  #rename the function for easier writing

trainX = enc.fit_transform(trainX)  #the output is numpy array (ndarray)

trainX = pd.DataFrame(trainX, columns=colnames) #convert ndarray to pandas data frame 
                                                #not required but good for printing 

trainX.head()  #sanity check

,Age,Sex,ChestPain,RestBP,Chol,Fbs,RestECG,MaxHR,ExAng,Oldpeak,Slope,Ca,Thal
0,31.0,1.0,0.0,7.0,63.0,0.0,2.0,48.0,0.0,6.0,0.0,2.0,0.0
1,5.0,1.0,0.0,12.0,39.0,0.0,0.0,31.0,0.0,12.0,1.0,0.0,2.0
2,37.0,0.0,0.0,8.0,3.0,0.0,0.0,20.0,0.0,16.0,1.0,0.0,1.0
3,9.0,0.0,0.0,22.0,118.0,1.0,2.0,28.0,1.0,27.0,1.0,0.0,2.0
4,18.0,1.0,3.0,12.0,16.0,0.0,2.0,73.0,0.0,0.0,1.0,0.0,0.0


#### Creating and fitting the model

We use `CategoricalNB()` function from `sklearn.naive_bayes` submodule, which requires encoding, which is why we did the previous step.

In [19]:
model = CategoricalNB()  #create model object
model.fit(trainX,trainBrnli) # fit on train data

CategoricalNB()

#### Predicting

To measure performance, we should use test data. That would be your job in a moment. However, here, we make predictions for each training observation. Again, this is not as reliable estimate of true performance as when test data are used. The main purpose using train rather than test data is is so you can use the code and modify it appropriately, in order to make predictions on test data.

**Note**: The reason why estimating performance using train data is not as reliable as when we use test data, is because the model was fitted in order to please the train data, i.e. to minimize the error (or equivalently, maximize performance) when the model is applied on train data. However, if the model goes closely around train data (i.e. fits well the train data), that doesn't necessarily mean it will be as close to some other data (recall the **bias-variance tradeoff** phenomenon). On the contrary, as bias-variance tradeoff suggests: in general, the more easily the model fits any given sample (i.e. the more flexible the model is), the more it moves away from other possible samples, and thus, performs poorly on other possible data sets. 

In [20]:
yhattrain = model.predict(trainX) # predict on train data

#### Confusion Matrix

To assess performance, we can take a look at confusion matrix.

We show confusion matrix for predictions made on train data, but you will do it the more appropriate way, i.e. using the test data.

In [21]:
## confusion matrix using pandas method crosstab
pd.crosstab(yhattrain, trainy)


AHD,No,Yes
row_0,,
0,103,11
1,8,86


In [22]:
#alternative way, which requires encoded labels, i.e. trainBrnli
confusion_matrix(yhattrain, trainBrnli)


array([[103,  11],
       [  8,  86]])

Note that the accuracy score, i.e. the proportion of correctly predicted outputs is ${103+86 \over103+11+8+86} \approx 90.87\%$. 

**Caveat!!!** Due to bias-variance tradeoff, we know that this relatively high score does not necessarily guarantee a good prediction on new (out-of-sample) data. It might be just due to overfitting - i.e. if the model is too flexible, it will easily fit the train data very well. Again, good performance on out-of-sample data are not guaranteed. This is the reason why we want to test performance on new data (test data), that were not used for fitting the model.

The accuracy score can also be computed using `accuracy_score()` function from `sklearn.matrics` module.

In [23]:
accuracy_score(yhattrain, trainBrnli)

0.9086538461538461

<br>

**Now it's your turn**

Hopefully, you correctly created `testX` and `testy` above, by writing something like

`testX = test_set.iloc[:,:-1]`

``testy = test_set['AHD']``

<br>





### <font style="color: red">Question 2.</font>

<br>

Now, it's your turn to repeat some parts of the procedure above, but for the test data, which is the right way. 

Do the following steps:

- Create `testBrnli` by 0-1 encoding `testy` variable

- Transform `testX` into pandas data frame by with encoded values of all the features. The transformed data frame should still be called `testX`.

- Create vector (i.e. numpy arrray) of predictions, called `yhattest`, based on the feature values from the test data. 


If everything is fine, first couple of values of `yhattest` should be `1, 0, 0, 1, 1, 0, 0, 1, 0`.



In [24]:
testBrnli = le.fit_transform(testy)
testBrnli[:5]
testX = enc.fit_transform(testX)
testX = pd.DataFrame(testX, columns=colnames) 
yhattest = model.predict(testX)
yhattest[:9]

array([1, 0, 0, 1, 1, 0, 0, 1, 0])

In [25]:
## test whether first 4 values of column testBrnli are correct
assert sum(testBrnli[:4] != [1, 0, 0, 0]) == 0

## test whether first 9 values of column yhattest are correct
assert sum(yhattest[:9] != [1, 0, 0, 1, 1, 0, 0, 1, 0]) == 0


### <font style="color: red">Question 3.</font>

<br>

Compute the confusion matrix (for test data) and store it as a variable `confM`.

**Hint**: the entry in the 1st row and 1st column (predicted value=No, actual AHD value=No) should be 42.

In [26]:
confM = confusion_matrix(yhattest, testBrnli)
confM

array([[42,  7],
       [ 7, 33]])

In [27]:
## check whether the determinant of the confusion matrix is 1337

assert abs(np.linalg.det(np.array(confM)) - 1337) < 1.e-5


In [28]:
## check whether the answer is correct (hidden test)



### <font style="color: red">Question 4.</font>

#### Accuracy Score

Compute accuracy score, i.e. proportion of the correctly predicted outputs. Store this value as python variable `acc`. So, your answer should look like

`acc = <some expression>`

Do **NOT** round your answer.

In [29]:
acc = accuracy_score(yhattest, testBrnli)

In [30]:
## test whether 5th and 6th decimal digits of acc are 96

assert np.mod(int(np.floor(acc * 10**6)), 100) == 96

In [31]:
## test whether acc is correct (hidden tests)




**Remark**

If you got `acc` right, you will notice that the accuracy score for prediction on test data is lower than the one for prediction on train data. As mentioned earlier, this is not surprising since the model is fitted on training data so that it maximizes performance (according to some criterion similar to accuracy score). When we move away from the training data and use other set (test set) to test performance, we get a more reliable estimate of the true expected accuracy score (accross all possible samples). 

While it is possible (depending on randomization of training and test samples) that the accuracy score happens to be higher when measured on test data than on train data, on average, it is expected to be lower on the test data. Also, on average, accuracy score measured on test data is more reliable estimate of the true expected accuracy score accross all possible samples. 

#### False Negative

We would now like to estimate the probability that the model does not detect heart disease on a patient that does have it (which can be devastating for the patient, if they don't treat it as soon as possible). This probability is called *false negative rate*, or *miss rate*.

In other words, we want to estimate the probability that the model will incorrectly predict $\widehat{AHD}=No$ in the situation when the patient does have heart disease, i.e. when actually $AHD=Yes$. In other words, we would like to estimate 

$$P(\widehat{AHD}=\text{No}|AHD = \text{Yes}) = P(\hat{y}=0\,| \, y=1)$$

The most natural way to estimate this is to use confusion matrix and among all true positives (i.e. among all the data with $y=1$), we find the proportion of those for which $\hat{y}=0$. In other words, we compute the ratio 

$\displaystyle {\#\{y=1 \text{ and } \hat{y}=0\}\over \#\{y=1\}} = {\text{all test data with } y=1 \text{ for which the model predicted } \hat{y}=0 \over \text{all test data with } y=1}$

<br>

<br>

### <font style="color: red">Question 5.</font>

Estimate the probability of false negative rate by the ratio given above. Use the test data only, i.e. use appropriate values from your confusion matrix `confM`. Store the estimate of the false negative rate as `FNR`. So, your answer should be of the form 

`FNR = <some expression>`

Do **NOT** round your answer.

<br>

In [32]:
FNR = 7/(7+33)

In [33]:
## test whether the 2nd decimal digit is 7

assert np.mod(int(np.floor(FNR * 10**2)), 10) == 7

In [34]:
## test whether the FNR is correct (hidden tests)

